In [0]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import pickle

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [3]:
from google.colab import files
uploaded = files.upload()

Saving test_label_out_r.pickle to test_label_out_r.pickle


In [5]:
from google.colab import files
uploaded = files.upload()

Saving 345294.965701808.png to 345294.965701808.png
Saving 345295.692871652.png to 345295.692871652.png
Saving 345296.417209573.png to 345296.417209573.png
Saving 345297.140051842.png to 345297.140051842.png
Saving 345297.861880295.png to 345297.861880295.png
Saving 345298.582676145.png to 345298.582676145.png
Saving 345299.305534548.png to 345299.305534548.png
Saving 345300.027744614.png to 345300.027744614.png
Saving 345300.755423811.png to 345300.755423811.png
Saving 345301.479338773.png to 345301.479338773.png
Saving 345302.203621029.png to 345302.203621029.png
Saving 345302.935515707.png to 345302.935515707.png
Saving 345303.664170098.png to 345303.664170098.png
Saving 345304.395943251.png to 345304.395943251.png
Saving 345305.125315681.png to 345305.125315681.png
Saving 345305.852504243.png to 345305.852504243.png


In [6]:
ls

345294.965701808.png  345299.305534548.png  345303.664170098.png
345295.692871652.png  345300.027744614.png  345304.395943251.png
345296.417209573.png  345300.755423811.png  345305.125315681.png
345297.140051842.png  345301.479338773.png  345305.852504243.png
345297.861880295.png  345302.203621029.png  sample_data/
345298.582676145.png  345302.935515707.png  test_label_out_r.pickle


In [0]:
def onehot(vals, possible_vals):
    if not isinstance(possible_vals, list): raise TypeError("provide possible_vals as a list")
    enc_vals = np.zeros([len(vals), len(possible_vals)])
    for i, value in enumerate(vals):
        if isinstance(possible_vals[0], float):
            enc = np.where(abs(possible_vals-value)<1e-3)
        else:
            enc = np.where(possible_vals==value)
        enc_vals[i,enc] = 1
    return enc_vals

In [0]:
class Rescale(object):
    def __init__(self, scalar):
        self.scalar = scalar

    def __call__(self, im):
        w, h = [int(s*self.scalar) for s in im.size]
        return transforms.Resize((h, w))(im)

class Crop(object):
    def __init__(self, box):
        assert len(box) == 4
        self.box = box

    def __call__(self, im):
        return im.crop(self.box)

class Augment(object):
    def __init__(self, seq):
        self.seq = seq

    def __call__(self, im):
        return Image.fromarray(self.seq.augment_images([np.array(im)])[0])

def get_data_transforms(t='train'):
    data_transforms = {
        'train': transforms.Compose([Crop((0,120,800,480)),
            Rescale(0.4),
            transforms.ToTensor()
        ]),
        'val': transforms.Compose([
            Crop((0,120,800,480)),
            Rescale(0.4),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    return data_transforms[t]

In [0]:
from torch.utils.data import Dataset
from matplotlib import image
from matplotlib import pyplot
from torchvision import transforms
from PIL import Image
#LABEL_KEYS = ['v_throttle', 'v_break', 'v_steer',
              #'traffic_light', 'front_vehicle', 'centre_dist', 'pedestrian_distance']

class CARLA_Dataset(Dataset):
    def __init__(self, t, pickle_file_path, image_dir_path):
        assert t in ['train', 'val']
        self.transform = get_data_transforms(t)
        #mention path containing training images
        self.image_list = os.listdir()
        self.image_list.remove('.config')
        self.image_list.remove('sample_data')
        #self.image_list.remove('test')
        self.image_list.remove('test_label_out_r.pickle')
        self.file_name = []
        for file in self.image_list:
            self.file_name.append(file[0:16])   
        #load the labels from pickle file
        self.labels = {}
        f = open('test_label_out_r.pickle', 'rb')  
        self.pickled_data = pickle.load(f)
        # transform reg affordances
        reg_keys = ['front_vehicle', 'centre_dist', 'pedestrian_distance', 'v_steer']
        for key in self.pickled_data:
            entry = self.pickled_data[key]
            #print("1.", entry['front_vehicle'])
        #reg_norm = entry[reg_keys] / abs(entry[reg_keys]).max()
                # transform cls affordances
      
        f.close()     

    def __len__(self):
        return len(self.file_name)

    def __getitem__(self, idx):
        current_fname = self.file_name[idx]
        self.raw_image = Image.open(current_fname + '.png')
        label_dict = self.pickled_data[current_fname]
        self.labels['traffic_light'] = torch.Tensor(onehot(np.array(label_dict['traffic_light']), [False, True]))
        #self.labels['v_break'] = torch.Tensor(onehot(np.array(list(label_dict['v_break'])), [0.0, 1.0]))
        print(label_dict['traffic_light'])
        #print(self.labels['traffic_light'])
        print(label_dict['v_break'])
        #print(self.labels['v_break'])
        #pyplot.imshow(self.raw_image)
        #pyplot.show()
        im = self.transform(self.raw_image)
        #print(im)
        label_values = list(label_dict.values())
        #print(label_values)
        print(label_dict)
        #reg_keys = ('front_vehicle', 'centre_dist', 'pedestrian_distance', 'v_steer')
        #reg_norm = label_dict[reg_keys] / abs(label_dict[reg_keys]).max()
        #reg_norm = label_dict[reg_keys]

        #for i in 'front_vehicle', 'centre_dist', 'pedestrian_distance', 'v_steer':
        #  reg_norm = label_dict[i] / abs(label_dict[i])
        #  print(i, reg_norm)

        #print(type(reg_keys))
        #reg_norm = label_dict['front_vehicle', 'centre_dist'] / max(abs(label_dict['front_vehicle', 'centre_dist']))
        sample = {'image' : self.raw_image, 'labels' : label_values}
        #sample = {'image' : self.raw_image}

        return sample
        #return label_dict

In [76]:
C_dataset = CARLA_Dataset( 'train', pickle_file_path='',
                                    image_dir_path='')

for i in range(len(C_dataset)):
  sample = C_dataset[i]
  #print(sample['image'])
  #print(sample['labels'])

(False, 'Green')
1.0
{'front_vehicle': 9.747201091003234, 'centre_dist': 0.47582735058322667, 'v_break': 1.0, 'pedestrian_distance': 50, 'v_steer': -0.0003462655295152217, 'traffic_light': (False, 'Green'), 'v_throttle': 0.0}
(False, 'Green')
0.0
{'front_vehicle': 10.842432177164572, 'centre_dist': 0.4761505371560434, 'v_break': 0.0, 'pedestrian_distance': 50, 'v_steer': -0.0002949305926449597, 'traffic_light': (False, 'Green'), 'v_throttle': 1.0}
(False, 'Green')
0.0
{'front_vehicle': 11.481087739082962, 'centre_dist': 0.47623467747072357, 'v_break': 0.0, 'pedestrian_distance': 50, 'v_steer': -0.00021634783479385078, 'traffic_light': (False, 'Green'), 'v_throttle': 1.0}
(False, 'Green')
1.0
{'front_vehicle': 8.287191490497623, 'centre_dist': 0.4753578950081617, 'v_break': 1.0, 'pedestrian_distance': 50, 'v_steer': -0.000217437744140625, 'traffic_light': (False, 'Green'), 'v_throttle': 0.0}
(False, 'Green')
1.0
{'front_vehicle': 8.84822808565823, 'centre_dist': 0.47553901514416974, 'v_

In [36]:
print(len(C_dataset))

16
